In [2]:
import pandas as pd
import sys, os
project_root = os.path.abspath(os.path.join(os.getcwd(), "../"))
sys.path.append(project_root)
from src.vectorStore import VectorStoreManager

✓ HuggingFace cache set to: c:\Users\Birhanu Matebe\Downloads\KAIM\RAG-Powered\RAG-Powered-Chatbot\rag-compliant-chatbot\models


In [3]:
df =pd.read_csv("../data/processed/complaints_processed_for_rag.csv")

In [ ]:
# 1. Initialize the manager (This sets the HF cache and loads the model)
vector_manager = VectorStoreManager(model_name='all-MiniLM-L6-v2')

# 2. Create the Stratified Sample (10,000 - 15,000 as per Task 2)
# df_cleaned is the result from your Task 1 Processor
sampled_df = vector_manager.prepare_stratified_sample(df, target_size=12000)

# 3. Build the Index (Chunking -> Embedding -> Indexing)
vector_db = vector_manager.build_and_save_index(sampled_df)

Initializing Embedding Model: all-MiniLM-L6-v2...


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


Creating stratified sample of 12000 records...


c:\Users\Birhanu Matebe\Downloads\KAIM\RAG-Powered\RAG-Powered-Chatbot\rag-compliant-chatbot\src\vectorStore.py:43: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby('Product', group_keys=False).apply(


Chunking 11999 narratives...
Generating embeddings for 20400 chunks (this may take a few minutes)...
✅ Vector store saved to ../vector_store/complaints_index


In [6]:
# Quick Test Search
test_query = "problems with unauthorized transactions on my credit card"
search_results = vector_db.similarity_search(test_query, k=2)

for i, doc in enumerate(search_results):
    print(f"\n--- Result {i+1} ---")
    print(f"Product: {doc.metadata['product']}")
    print(f"Complaint ID: {doc.metadata['complaint_id']}")
    print(f"Text Chunk: {doc.page_content[:200]}...")


--- Result 1 ---
Product: Credit card
Complaint ID: 12483979
Text Chunk: credit card identify steal unauthorized transaction numerous place fast amount time dispute transaction discover come back say fast enough report false call freeze credit card say pay item never purch...

--- Result 2 ---
Product: Credit card
Complaint ID: 7826217
Text Chunk: fraudulent activity discover credit card...


## Text Chunking, Embedding, and Vector Store Indexing

Sampling Strategy: "A stratified sample of 12,000 records was successfully extracted, ensuring proportional representation across all product categories."

Chunking Approach: "We implemented a RecursiveCharacterTextSplitter with a chunk size of 600 and an overlap of 60. This preserves local context while optimizing for the 384-dimension limit of the embedding model."

Embedding & Indexing: "The all-MiniLM-L6-v2 model was used to generate embeddings. The resulting vectors and their associated metadata (ID, Product, Issue) were persisted locally using a FAISS index, as evidenced by the generated index.faiss and index.pkl files."